In [6]:
!pip install pygame

In [7]:
import numpy as np
import pygame as p
import sys
import random
import copy

In [8]:
# Constants:
WIDTH, HEIGHT = 400, 400
ROWS, COLS, = 8, 8
BOX_SIZE = WIDTH//COLS
RED, GREEN, BLUE = (255,0,0), (0,255,0), (0,0,255)
WHITE, BLACK, SILVER = (255, 255, 255), (0,0,0), (192,192,192)
SALMON = (252,151,151)
DARK =  (184,139,74)
LIGHT = (227,193,111)

image_dimensions = int(17/400 * WIDTH)
CROWN = p.transform.scale(p.image.load("crown.png"), (image_dimensions,image_dimensions))

In [9]:
class Piece:

    PADDING = 14
    BORDER = 2

    def __init__(self, row, column, colour):
        self.row = row
        self.column = column
        self.colour = colour
        if self.colour == WHITE:
            self.direction = 1
        else:
            self.direction = -1
        self.king = False
        self.x = 0
        self.y = 0
        self.calculate_position()

    def set_king(self):
        self.king = True

    def calculate_position(self):
        self.x = BOX_SIZE * self.column + BOX_SIZE // 2
        self.y = BOX_SIZE * self.row + BOX_SIZE // 2

    def draw(self, win):
        radius = BOX_SIZE//2 - self.PADDING
        p.draw.circle(win, SILVER, (self.x,self.y), radius + self.BORDER)
        p.draw.circle(win, self.colour, (self.x,self.y), radius)
        if self.king:
            new_x = self.x - CROWN.get_width()//2
            new_y = self.y - CROWN.get_height()//2
            win.blit(CROWN, (new_x, new_y))

    def move_piece(self, row, column):
        self.row = row
        self.column = column
        self.calculate_position()

    def __repr__(self):
        if self.direction > 0:
            return "+"
        else:
            return "-"

In [10]:
class GameBoard:


    def __init__(self):
        self.board = np.zeros(shape=(8,8)).astype("int").tolist()
        self.player_count = 0
        self.red_remaining = self.white_remaining = 12
        self.red_king_count = self.white_king_count = 0
        self.setup_board()


    def setup_board(self):
        # Start player 2 at "top" of board.
        colour = 2
        # Counter to switch to using player 1 identifier.
        counter = 0 
        ln = len(self.board)
        for row_index in range(ln):
          if self.is_even(row_index) and row_index != 4:
            self.board[row_index] = [colour,0,colour,0,colour,0,colour,0]
          elif not self.is_even(row_index) and row_index != 3:
            self.board[row_index] = [0,colour,0,colour,0,colour,0,colour]
          counter += 1
          if counter == 4:
            colour = 1
        # Messy Setup.
        for row_index in range(ln):
          for col_index in range(ln):
            if self.board[row_index][col_index] != 0:
              colour = self.board[row_index][col_index]
              if colour == 1: 
                colour = RED
              else: 
                colour = WHITE
              self.board[row_index][col_index] = Piece(row_index, col_index, colour) 


    def draw_board(self, win):
        win.fill(LIGHT)
        for row in range(ROWS):
          for col in range(row % 2, COLS, 2):
            p.draw.rect(win, DARK, (row*BOX_SIZE, col*BOX_SIZE, BOX_SIZE, BOX_SIZE))


    def draw_all(self, win):
        # Draw board.
        self.draw_board(win)
        # Draw pieces on board.
        for row in range(ROWS):
          for col in range(COLS):
            piece = self.board[row][col]
            if piece != 0:
              piece.draw(win)


    def get_valid_moves(self, piece):
        full_moves = []
        if piece.king:
            for direction in [-1,+1]:
                full_moves.append(self.get_valid_moves_dir(piece,direction))
        else:
            full_moves.append(self.get_valid_moves_dir(piece,piece.direction))
        full_moves = self.flatten(full_moves)
        for move in full_moves:
            if self.can_capture(piece.row, move[0]):
                full_moves = [move for move in full_moves if self.can_capture(piece.row, move[0])]
                break
        
        return full_moves

    def get_valid_moves_dir(self, piece, direction):
        # End function if non player piece.
        try:
          # Get row, column (indicies) from tuple object piece.
          row, column = piece.row, piece.column
        except AttributeError:
            print("No valid moves for an empty space!")
            return
        # Potential next moves list.
        next_move_list = []
        # Next row - dependent on player.
        next_row = row + direction
        next_next_row = row + direction + direction
        # List to hold columns on either side.
        left_right = [column-1,column+1]
        # Loop through left right options.
        for next_col in left_right:
          if next_col in range(8) and next_row in range(8):
            next_space = self.whats_in_the_box(next_row, next_col)
            if isinstance(next_space, Piece):
              # Split conditional - case next_space=0, no int attribute colour.
              if next_space.colour != piece.colour:
              # Assign next next column indicies.
                next_next_col = None
                if next_col == column - 1:
                    next_next_col = column - 2
                else:
                    next_next_col = column + 2
                if next_next_col in range(8) and next_next_row in range(8):
                    if self.whats_in_the_box(next_next_row,next_next_col) == 0:
                        next_move_list.append((next_next_row, next_next_col))
        # If no forced capture moves yet.
        if not next_move_list:
          # Case: Empty square.
          for next_col in left_right:
            if next_col in range(8) and next_row in range(8):
              # Check state of potential next square.
              if self.whats_in_the_box(next_row, next_col) == 0:
                next_move_list.append((next_row, next_col))
        return next_move_list


    def can_capture(self, row, new_row):
        if new_row in [row+2, row-2]:
            return True
        return False


    def move_piece(self, Piece, new_row, new_col):
        # Temp. remove (parameter) Piece.
        row, col = Piece.row, Piece.column
        self.remove_piece(row,col)
        Piece.move_piece(new_row, new_col)
        self.board[new_row][new_col] = Piece
        # King update.
        if (new_row == 0 or new_row == ROWS - 1) and Piece.king != True:
            Piece.set_king()
            if Piece.colour == RED:
                self.red_king_count += 1
            else:
                self.white_king_count += 1
        # Remove opponent piece:
        if new_row in [row+2, row-2]:
            x = (row+new_row)//2
            y = (col+new_col)//2
            opp_piece = self.whats_in_the_box(x,y)
            if opp_piece.colour == RED:
                self.red_remaining -= 1
                if opp_piece.king:
                    Piece.set_king()
                    self.white_king_count += 1
                    self.red_king_count -= 1
            else:
                self.white_remaining -=1
                if opp_piece.king:
                    Piece.set_king()
                    self.red_king_count += 1
                    self.white_king_count -= 1
            self.remove_piece(x,y)
              # Return True if piece is taken.
            return True
        # Else False
        return False


    def whats_in_the_box(self, row, column):
        return self.board[row][column]

    def remove_piece(self, row, col):
        self.board[row][col] = 0

    def get_all_pieces(self, colour, capture=True):
        total = []
        for row in self.board:
            for item in row:
                if isinstance(item, Piece) and item.colour == colour:
                    total.append(item)
        if capture:
            pieces_can_capture = []
            for piece in total:
                for move in self.get_valid_moves(piece):
                    if move:
                        if self.can_capture(piece.row, move[0]):
                            pieces_can_capture.append(piece)
            if pieces_can_capture:
                return pieces_can_capture
        return total
    
    
    def evaluate(self, colour):
        if colour == WHITE:
            return self.white_remaining - self.red_remaining + (self.white_king_count * 0.5 - self.red_king_count * 0.5)

        else:
            return self.red_remaining - self.white_remaining + (self.red_king_count * 0.5 - self.white_king_count * 0.5)

        
    def can_colour_move(self, colour):
        pieces = self.get_all_pieces(colour, capture=False)
        can_move = False
        for piece in pieces:
            if self.get_valid_moves(piece):
                can_move = True
                break
        return can_move
        
    def gameover(self):
        if self.red_remaining == 0 or not self.can_colour_move(RED):
            return WHITE
        elif self.white_remaining == 0 or not self.can_colour_move(WHITE):
            return RED
        else:
            return False

    def is_even(self,num):
        return (num % 2) == 0 
    
    def flatten(self,ls):
        return [item for m_ls in ls for item in m_ls]
        
    def print_board(self):
        print()
        for row in self.board:
            print(row)
        print()

In [11]:
gb = GameBoard()
gb.setup_board()
gb.print_board()
white = gb.whats_in_the_box(2,6)
gb.move_piece(white, 7, 3)
# red = gb.whats_in_the_box(5,7)
# gb.move_piece(red, 4, 6)
# red.set_king()
# gb.remove_piece(7,3)
gb.remove_piece(5,1)
gb.remove_piece(5,5)
# gb.remove_piece(6,4)
gb.print_board()
print()
print(gb.get_valid_moves(white))

# print(len(gb.get_all_pieces(RED)))
# test_gb.can_capture(test_gb.board[2][0], test_gb.board[2][0].direction)


[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, -, 0, -, 0, -]


[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -, 0, 0, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, +, 0, -, 0, -]


[(5, 1), (5, 5)]


In [12]:
class GameManager:

    def __init__(self, win, scorepanel_size, difficulty=1):
        self.win = win
        self.scorepanel_size = scorepanel_size
        self.difficulty = difficulty
        self.__init()

    # Hidden method to force reset_game() call.
    def __init(self):
        self.selected_piece = None
        self.gameboard = GameBoard()
        self.valid_moves = []
        self.turn = RED
        self.hint_squares = []
        self.show_hint = False
        self.can_capture_pieces = []
        self.correct_moves_assist = False

    def reset_game(self):
        self.__init()

    def update(self):
        self.gameboard.draw_all(self.win)
        self.draw_valid_moves(self.valid_moves)
        self.scorepanel()
        self.draw_hints()
        if self.correct_moves_assist:
            self.draw_correct_moves()
        p.display.update()
        p.display.flip()
    
        
    def select_piece(self, row, col):
        # Allow for continous selection of pieces.
        if self.selected_piece:
            self.show_hint = False
            b = self.move_piece(row, col)
            if not b:
                # Reset selection.
                self.selected_piece = None
                self.select_piece(row,col)
        piece = self.gameboard.whats_in_the_box(row, col)
        if piece.colour == self.turn:
            self.selected_piece = piece
            self.valid_moves = self.gameboard.get_valid_moves(piece)
            return True
        return False
            

    def move_piece(self, row, col): 
        self.can_capture_pieces = self.gameboard.get_all_pieces(self.turn,capture=True)
        piece = self.gameboard.whats_in_the_box(row, col)
        # If selected, piece is not a piece, (row,col) is valid move then move.
        if self.selected_piece and not isinstance(piece, Piece) and (row, col) in self.valid_moves:
            if self.selected_piece in self.can_capture_pieces:
                self.gameboard.move_piece(self.selected_piece, row, col)
                # If move is successful switch current turn.
                self.turn_switch()
            else:
                self.correct_moves_assist = True
                return False
        else:
            return False
        return True

    
    def turn_switch(self):
        self.can_capture_pieces = []
        self.correct_moves_assist = False
        self.hint_squares = []
        self.valid_moves = []
        if self.turn == RED:
            self.turn = WHITE
        else:
            self.turn = RED
            
    def turn_ai(self, board):
        self.gameboard = board
        self.turn_switch()
        
    def get_board(self):
        return self.gameboard


    def draw_valid_moves(self, valid_moves):
        for pos_move in valid_moves:
            row, col = pos_move
            circle_x = col*BOX_SIZE + BOX_SIZE//2
            circle_y = row*BOX_SIZE + BOX_SIZE//2
            colour = None
            if self.turn == RED:
                colour = SALMON
            else:
                colour = SILVER
            p.draw.circle(self.win, colour, (circle_x, circle_y), 10)
            
    def draw_hints(self):
        if self.show_hint:
            row, col = self.hint_squares[0][0], self.hint_squares[0][1]
            p.draw.rect(self.win, GREEN, (col*BOX_SIZE, row*BOX_SIZE, BOX_SIZE, BOX_SIZE), width=3)
            # Suggested move.
            row, col = self.hint_squares[1][0], self.hint_squares[1][1]
            p.draw.rect(self.win, GREEN, (col*BOX_SIZE, row*BOX_SIZE, BOX_SIZE, BOX_SIZE), width=3)    
        
    def draw_correct_moves(self):
        for piece in self.can_capture_pieces:
            p.draw.rect(self.win, BLUE, (piece.column*BOX_SIZE, piece.row*BOX_SIZE, BOX_SIZE, BOX_SIZE), width=3)
            
    def scorepanel(self):
        # Panel shape.
        p.draw.rect(self.win, WHITE, (WIDTH,0,self.scorepanel_size-2,HEIGHT))
        p.draw.rect(self.win, BLACK, (WIDTH,0,self.scorepanel_size-2,HEIGHT), self.scorepanel_size//20)
        # Turn marker.
        self.scorepanel_turn_marker()
        # Restart button.
        self.scorepanel_button("RESTART", HEIGHT-100)
        # Quit button.
        self.scorepanel_button("QUIT", HEIGHT-50)
        # Hint button.
        self.scorepanel_button("HINT", HEIGHT - 150)
        # Difficulty button.
        self.scorepanel_button("DIFFICULTY", HEIGHT - 200, text_size=12)
        # Invalid Move message.
        if self.correct_moves_assist:
            self.scorepanel_button("FORCED CAPTURE", HEIGHT-250, text_size=13, text_colour=BLUE, rect_colour=WHITE)
        # Debugging.
#         self.scorepanel_button(str(self.valid_moves), HEIGHT - 250)
        # Pieces Remaining + Difficulty level.
        self.scorepanel_info_text()
        
    def scorepanel_turn_marker(self):
        txt = None
        circle_colour = None
        txt_colour = BLACK
        if not self.gameboard.gameover():
            txt = "TURN"
            circle_colour = self.turn
        else:
            txt = "WINNER"
            txt_colour = GREEN
            circle_colour = self.gameboard.gameover()
        circle_x, circle_y = WIDTH + (self.scorepanel_size/2), 50
        self.scorepanel_button(txt, circle_y-45, 23, txt_colour, WHITE, 0)
        if circle_colour == RED:
            p.draw.circle(self.win, RED, (circle_x, circle_y), 20)
        else: 
            p.draw.circle(self.win, BLACK, (circle_x, circle_y), 20, 2)
        
    def scorepanel_button(self, button_text, y_pos, text_size=16, text_colour=RED, rect_colour=BLACK, border=5):
        smallfont = p.font.SysFont('Corbel',text_size) 
        text = smallfont.render(button_text , True , text_colour)
        centre = WIDTH + self.scorepanel_size/2
        displacement = 30
        p.draw.rect(self.win, rect_colour, (centre - displacement, y_pos,displacement*2,30), border)
        text_rect = text.get_rect(center=(centre, y_pos+(displacement//2)))
        self.win.blit(text, text_rect)
        
    def scorepanel_info_text(self):
        red_rem = "Red Pieces: {}".format(self.gameboard.red_remaining)
        white_rem = "White Pieces: {}".format(self.gameboard.white_remaining)
        self.scorepanel_button(white_rem,100, 15, BLACK, WHITE)
        self.scorepanel_button(red_rem,75, 15, BLACK, WHITE)
        difficulty_dict = {-1:"No AI",
                            0:"Easy",
                            1:"Medium",
                            2:"Hard",
                            3:"Very Hard",
                            4:"Last Stand"}
        difficulty_text = "Difficulty: {}".format(difficulty_dict[self.difficulty])
        self.scorepanel_button(difficulty_text, 125, 14, BLACK, WHITE)

    def set_difficulty(self, difficulty):
        self.difficulty = difficulty
        
    def set_hint_squares(self, hint_list):
        if self.show_hint == False:
            self.show_hint = True
        else:
            self.show_hint = False
        self.hint_squares = hint_list


In [13]:
class AI_Player:
    
    def __init__(self, gamemanager, difficulty=1):
        self.difficulty = difficulty
        self.gamemanager = gamemanager
        self.recursive_calls = 0
        
    def update_gamemanager(self, gamemanager):
        self.gamemanager = gamemanager
        
    def set_difficulty(self, difficulty):
        self.difficulty = difficulty
        
    def reset_recursive_calls(self):
        self.recursive_calls = 0
        
    def hint_move(self):
        player1 = None
        player2 = None
        if self.gamemanager.turn == RED:
            player1 = RED
            player2 = WHITE
        else:
            player1 = WHITE
            player2 = RED
        original_board = self.gamemanager.get_board()
        _, new_board = self.minimax(original_board, 2, True, self.gamemanager,
                                    maxi_colour=player1, mini_colour=player2)
        original_pieces = original_board.get_all_pieces(player1, capture=False)
        new_pieces = new_board.get_all_pieces(player1, capture=False)
        original_positions_list = []
        new_positions_list = []
        for _, (original_piece,new_piece) in enumerate(zip(original_pieces,new_pieces)):
            original_positions_list.append((original_piece.row, original_piece.column))
            new_positions_list.append((new_piece.row,new_piece.column))
        original_position = None
        new_position = None
        for position in original_positions_list:
            if position not in new_positions_list:
                original_position = position
                break
        for position in new_positions_list:
            if position not in original_positions_list:
                new_position = position
                break
        return (original_position, new_position)
        
        
    def ai_move(self, print_calls=False):
        # Diff = -1 -> PvP
        # Diff = 0 -> Easy
        if self.difficulty == 0:
            new_board  = self.random_AI()
        # Diff = 1 -> Medium
        elif self.difficulty == 1:
            _, new_board = self.minimax(self.gamemanager.get_board(), 2, True, self.gamemanager)
        # Diff = 2 -> Hard
        elif self.difficulty == 2:
            _, new_board = self.minimax(self.gamemanager.get_board(), 5, True, self.gamemanager)
        # Diff = 3 -> Very Hard
        elif self.difficulty == 3:
            _, new_board = self.minimax(self.gamemanager.get_board(), 7, True, self.gamemanager)
        # Diff = 4 -> Last Stand - Progressively harder
        elif self.difficulty == 4:
            initial = 7
            add = 0
            opponent_remaining = len(self.gamemanager.gameboard.get_all_pieces(WHITE,capture=False))
            if opponent_remaining <= 3:
                add = 5
            elif opponent_remaining <= 5:
                add = 3
            elif opponent_remaining <= 8:
                add = 1
            _, new_board = self.minimax(self.gamemanager.get_board(), initial+add, True, self.gamemanager)
        if print_calls:
            print(self.recursive_calls)
        self.reset_recursive_calls()
        return new_board
    
    
    def random_AI(self, colour=WHITE):
        AI_pieces = self.gamemanager.gameboard.get_all_pieces(colour)
        AI_pieces = [x for x in AI_pieces if self.gamemanager.gameboard.get_valid_moves(x)]
        ls = list(range(len(AI_pieces)))
        random.shuffle(ls)
        r_num = ls[0]
        random_piece = AI_pieces[r_num]
        new_board = copy.deepcopy(self.gamemanager.gameboard)
        random_piece = new_board.board[random_piece.row][random_piece.column]
        moves = new_board.get_valid_moves(random_piece)
        random.shuffle(moves)
        new_row, new_col = moves[0][0], moves[0][1]
        new_board.move_piece(random_piece, new_row, new_col)
        return new_board
    
    
    
    def minimax(self, board, depth, maximiser, gamemanager,
                alpha=float("-inf"), beta=float("inf"),
                maxi_colour=WHITE, mini_colour=RED):
        self.recursive_calls += 1
        if depth == 0 or board.gameover():
            return board.evaluate(maxi_colour), board
        if maximiser:
            max_evaluation = float("-inf")
            best_move = None
            for move in self.get_all_moves(board, maxi_colour, gamemanager):
                # [0] to return only board evaluation value.
                evaluation = self.minimax(move, depth-1, False, gamemanager, alpha, beta,
                                          maxi_colour, mini_colour)[0]
                max_evaluation = max(max_evaluation, evaluation)
                if max_evaluation == evaluation:
                    best_move = move
                alpha = max(alpha, max_evaluation)
                if beta <= alpha:
                    break
            return max_evaluation, best_move
        else:
            min_evaluation = float("inf")
            best_move = None
            for move in self.get_all_moves(board, mini_colour, gamemanager):
                # [0] to return only board evaluation value.
                evaluation = self.minimax(move, depth-1, True, gamemanager, alpha, beta,
                                         maxi_colour, mini_colour)[0]
                min_evaluation = min(min_evaluation, evaluation)
                if min_evaluation == evaluation:
                    best_move = move
                beta = min(beta, min_evaluation)
                if beta <= alpha:
                    break
            return min_evaluation, best_move
    

    def simulate_moves(self, piece, move, board):
        new_row, new_col = move[0], move[1]
        board.move_piece(piece, new_row, new_col)
        return board

    def get_all_moves(self, board, colour, game):
        potential_boards_list = []
        potential_pieces = board.get_all_pieces(colour)
        for piece in board.get_all_pieces(colour):
            valid_moves = board.get_valid_moves(piece)
            for move in valid_moves:
                temp_board = copy.deepcopy(board)
                temp_piece = temp_board.whats_in_the_box(piece.row, piece.column)
                new_board = self.simulate_moves(temp_piece, move, temp_board)
                potential_boards_list.append(new_board)
        return potential_boards_list


In [14]:
def get_mouse_pos(pos):
    x, y = pos
    row = y // BOX_SIZE
    col = x // BOX_SIZE
    return row, col

def restart_game(pos, gamemanager):
    centre = WIDTH + (scorepanel_size/2)
    if centre - 30 <= pos[0] <= centre + 30 and HEIGHT - 100 <= pos[1] <= HEIGHT - 70:
        gamemanager.reset_game()

def quit_game(pos):
    centre = WIDTH + (scorepanel_size/2)
    if centre - 30 <= pos[0] <= centre + 30 and HEIGHT - 50 <= pos[1] <= HEIGHT - 20:
        return False
    else:
        return True
    
def change_difficulty(pos, ai):
    centre = WIDTH + (scorepanel_size/2)
    if centre - 30 <= pos[0] <= centre + 30 and HEIGHT - 200 <= pos[1] <= HEIGHT - 170:
        new_diff = ai.difficulty + 1
        if new_diff not in range(-1,5):
            new_diff = -1
        ai.set_difficulty(new_diff)
        ai.gamemanager.set_difficulty(new_diff)
        
def hint(pos, ai):
    centre = WIDTH + (scorepanel_size/2)
    if centre - 30 <= pos[0] <= centre + 30 and HEIGHT - 150 <= pos[1] <= HEIGHT - 120:
        ai.gamemanager.set_hint_squares(ai.hint_move())

In [15]:
# Set size to 0 to play without scorepanel.
scorepanel_size = 110

p.display.init()
p.font.init()
SCREENSIZE = (WIDTH+scorepanel_size, HEIGHT)
WIN = p.display.set_mode(SCREENSIZE)
p.display.set_caption("DRAUGHTS")
p.mouse.set_cursor(*p.cursors.tri_left)
FPS = 30



def main():
    run = True
    clock = p.time.Clock()
    gm = GameManager(WIN, scorepanel_size)
    opponent = AI_Player(gm)
    AI_player = True
    
    
    while run:
        # Maintain constant frames/second.
        clock.tick(FPS)
        
        if gm.turn == WHITE and AI_player and not gm.gameboard.gameover() and opponent.difficulty != -1:
            opponent.update_gamemanager(gm)
            new_board = opponent.ai_move(True)
            gm.turn_ai(new_board)
        
        # Look for events during run.
        for event in p.event.get():
            # Non button quit:
            if event.type == p.QUIT:
                run = False
            # Mouse click events:
            if event.type == p.MOUSEBUTTONDOWN:
                pos = p.mouse.get_pos()
                row, col = get_mouse_pos(pos)
                try:
                    gm.select_piece(row,col)
                except:
                    pass
                # Restart Game.
                restart_game(pos, gm)
                # Quit Game.
                run = quit_game(pos)
                # Change difficulty.
                change_difficulty(pos, opponent)
                # Hint.
                hint(pos, opponent)

        
        gm.update()
        
    p.display.quit()

main()

22
477
3690
1069
652
4270
2503
8430
2973
3141
16616
26028
17926
110785
47927
58811
34598
210068
242278
161090
36856


### N.B.
* Threefold repitition: restriction
* [Randomness](https://www.reddit.com/r/gamedesign/comments/2mrqum/deterministic_vs_non_deterministic_games/) - determinisitic vs non deterministic games - Player inputs introduce randomness to keep game interesting.
* [Stackoverflow](https://stackoverflow.com/questions/20901882/best-ai-approach-for-game-draught-chekers)
* [Graphs](https://www.cs.huji.ac.il/~ai/projects/old/English-Draughts.pdf)
* [Good A-B pruning article](http://www.cs.columbia.edu/~devans/TIC/AB.html)
* [solved game](https://en.wikipedia.org/wiki/Solved_game)
* [solved game - paper](https://science.sciencemag.org/content/317/5844/1518#:~:text=Since%201989%2C%20almost%20continuously%2C%20dozens,sides%20leads%20to%20a%20draw.)
* [Machines that play checkers](https://hackernoon.com/machines-that-play-checkers-10f7d4038956)
* [Game Complexitiy - Ctrl f checkers](https://en.wikipedia.org/wiki/Game_complexity)

In [16]:
# import copy
# import random

# def random_AI(game):
#     AI_pieces = game.gameboard.get_all_pieces(WHITE)
#     AI_pieces = [x for x in AI_pieces if game.gameboard.get_valid_moves(x)]
#     ls = list(range(len(AI_pieces)))
#     random.shuffle(ls)
#     r_num = ls[0]
#     random_piece = AI_pieces[r_num]
#     new_board = copy.deepcopy(game.gameboard)
#     random_piece = new_board.board[random_piece.row][random_piece.column]
#     moves = new_board.get_valid_moves(random_piece)
#     random.shuffle(moves)
#     new_row, new_col = moves[0][0], moves[0][1]
#     new_board.move_piece(random_piece, new_row, new_col)
#     return new_board
        


# def minimax(board, depth, maximiser, gamemanager, alpha=float("-inf"), beta=float("inf")):
#     if depth == 0 or board.gameover():
#         return board.evaluate(), board
#     if maximiser:
#         max_evaluation = float("-inf")
#         best_move = None
#         for move in get_all_moves(board, WHITE, gamemanager):
#             # [0] to return only board evaluation value.
#             evaluation = minimax(move, depth-1, False, gamemanager, alpha, beta)[0]
#             max_evaluation = max(max_evaluation, evaluation)
#             if max_evaluation == evaluation:
#                 best_move = move
#             alpha = max(alpha, max_evaluation)
#             if beta <= alpha:
#                 break
#         return max_evaluation, best_move
#     else:
#         min_evaluation = float("inf")
#         best_move = None
#         for move in get_all_moves(board, RED, gamemanager):
#             # [0] to return only board evaluation value.
#             evaluation = minimax(move, depth-1, True, gamemanager, alpha, beta)[0]
#             min_evaluation = min(min_evaluation, evaluation)
#             if min_evaluation == evaluation:
#                 best_move = move
#             beta = min(beta, min_evaluation)
#             if beta <= alpha:
#                 break
#         return min_evaluation, best_move
    
    
# def simulate_moves(piece, move, board):
#     new_row, new_col = move[0], move[1]
#     board.move_piece(piece, new_row, new_col)
#     return board

# def get_all_moves(board, colour, game):
#     potential_boards_list = []
#     for piece in board.get_all_pieces(colour):
#         valid_moves = board.get_valid_moves(piece)
#         for move in valid_moves:
#             temp_board = copy.deepcopy(board)
#             temp_piece = temp_board.whats_in_the_box(piece.row, piece.column)
#             new_board = simulate_moves(temp_piece, move, temp_board)
#             potential_boards_list.append(new_board)
#     return potential_boards_list
    
    